In [7]:
import sklearn 
from sklearn.linear_model import LogisticRegression
import pandas as pd 
import numpy as np
from statsmodels.tsa.arima_model import ARMA
import datetime
import re

pd.options.display.max_rows = 999
pd.set_option('display.max_columns', 500)

In [11]:
DATA_PATH = '../ncaa_data/mens-machine-learning-competition-2019/'

# Load CSVs

In [12]:
regular_season_results = pd.read_csv(DATA_PATH + "RegularSeasonDetailedResults.csv")
tourney_results = pd.read_csv(DATA_PATH + "NCAATourneyDetailedResults.csv")
secondary_results = pd.read_csv(DATA_PATH + "SecondaryTourneyTeams.csv")

results = pd.concat([regular_season_results, tourney_results])

FileNotFoundError: File b'../ncaa_data/womens-machine-learning-competition-2019/WSecondaryTourneyTeams.csv' does not exist

In [13]:
seasons = pd.read_csv(DATA_PATH + "Seasons.csv")
seasons = seasons[["Season","DayZero"]]
results = results.set_index("Season").join(seasons.set_index("Season"), how="inner", rsuffix="season").reset_index()

NameError: name 'results' is not defined

In [6]:
results["WFGM_Perc"] = results["WFGM"]/results["WFGA"]
results["LFGM_Perc"] = results["LFGM"]/results["LFGA"]
results["WFGM3_Perc"] = results["WFGM3"]/results["WFGA3"]
results["LFGM3_Perc"] = results["LFGM3"]/results["LFGA3"]
results["WFTM_Perc"] = results["WFTM"]/results["WFTA"]
results["LFTM_Perc"] = results["LFTM"]/results["LFTA"]

NameError: name 'results' is not defined

In [67]:
# player_df = pd.read_csv(DATA_PATH + "Players_2010.csv")

# Build Team/Season Features

In [68]:
winning_results = results[['Season', 'DayNum','DayZero', 'WTeamID', 'WScore', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
'WAst', 'WTO', 'WStl', 'WBlk', 'WPF',"WFGM_Perc","WFGM3_Perc","WFTM_Perc"]]
losing_results = results[['Season', 'DayNum','DayZero', 'LTeamID', 'LScore', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF',"LFGM_Perc","LFGM3_Perc","LFTM_Perc"]]
winning_results.columns = ['Season', 'DayNum','DayZero', 'TeamID', 'Score','FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR',
'Ast', 'TO', 'Stl', 'Blk', 'PF',"FGM_Perc","FGM3_Perc","FTM_Perc"]
losing_results.columns = ['Season', 'DayNum','DayZero', 'TeamID', 'Score', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR',
'Ast', 'TO', 'Stl', 'Blk', 'PF',"WFGM_Perc","FGM3_Perc","FTM_Perc"]
team_df = pd.concat([winning_results, losing_results])


/anaconda3/envs/general-enrichment/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [69]:
seeds = pd.read_csv(DATA_PATH + "NCAATourneySeeds.csv")
seeds = seeds.set_index(["Season","TeamID"])
team_df = team_df.reset_index().join(seeds, how="inner", on=["Season","TeamID"])

In [70]:
team_df["Seed"] = team_df["Seed"].apply(lambda x:int(re.search(r'\d+', x).group()))
team_df["Seed"] = team_df["Seed"].fillna(20)

# Bring in Massey Ordinals 

In [71]:
massey = pd.read_csv(DATA_PATH + "MasseyOrdinals.csv")
massey = massey.rename(columns={"RankingDayNum":"DayNum"})
massey = massey.pivot_table(index=["Season","TeamID","DayNum"], columns="SystemName", values="OrdinalRank")
massey = massey.fillna(method="ffill")
massey = massey.drop("DC2", axis=1)
massey = massey.fillna(massey.mean())

In [72]:
team_df = team_df.set_index(["Season","TeamID","DayNum"]).join(massey, how="left")


In [73]:
team_df = team_df.reset_index().sort_values(["Season","DayNum"]).set_index("DayNum").groupby(["Season","TeamID"]).apply(lambda x: x.reindex(range(0,150), method='ffill').ffill())

# Clean Up and Make Some Columns MA

In [74]:
team_df = team_df.drop(["TeamID","Season"], axis=1)
deltas = team_df.reset_index()["DayNum"].apply(lambda x: datetime.timedelta(days=x))
team_df["date"] = pd.to_datetime(team_df["DayZero"]) + deltas.values

In [75]:
exclude = ["date","DayZero","Score","DayNum","Season"]
ma_columns = team_df.columns.tolist()
ma_columns = [item for item in ma_columns if item not in exclude]

In [76]:
ma_data = team_df.groupby(["Season","TeamID"])[ma_columns].apply(lambda x: x.rolling(window=20, min_periods=1).mean())

In [77]:
team_df[ma_columns] = ma_data

# Pickle Team DF

In [78]:
team_df.to_pickle(DATA_PATH + "team_df.p")

# Build Game DF

In [79]:
games = results[["WTeamID","LTeamID","Season","DayNum","WLoc","DayZero","WScore","LScore"]]

In [80]:
games["Team1"] = games[["WTeamID","LTeamID"]].min(axis=1)
games["Team2"] = games[["WTeamID","LTeamID"]].max(axis=1)


/anaconda3/envs/general-enrichment/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/envs/general-enrichment/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [81]:
games["Team1Score"] = 0
games["Team2Score"] = 0
games["Team1"] = games[["WTeamID","LTeamID"]].min(axis=1)
games["Team2"] = games[["WTeamID","LTeamID"]].max(axis=1)
games.loc[games["WTeamID"] == games["Team1"],"Team1Score"] = games["WScore"]
games.loc[games["WTeamID"] != games["Team1"],"Team1Score"] = games[games["WTeamID"] != games["Team1"]]["LScore"]
games.loc[games["WTeamID"] == games["Team2"],"Team2Score"] = games["WScore"]
games.loc[games["WTeamID"] != games["Team2"],"Team2Score"] = games["LScore"]
# games["ScoreDiff"] = 
games["Team1Home"] = (games["WLoc"]=="H") & (games["WTeamID"] == games["Team1"])
games["Team2Home"] = (games["WLoc"]=="H") & (games["WTeamID"] == games["Team2"])

/anaconda3/envs/general-enrichment/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/envs/general-enrichment/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/envs/general-enrichment/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [82]:
games = games.drop(["WTeamID","LTeamID","WLoc","DayZero"], axis=1)


# Increment the dayNum so that we only use results up through the day previous to the game

In [83]:
incremented = team_df.reset_index()
incremented["DayNum"] = incremented["DayNum"] + 1
incremented = incremented.set_index(["Season","TeamID","DayNum"])
full_df = games.join(incremented, on=["Season", "Team1","DayNum"], how="right").join(incremented, on=["Season", "Team2","DayNum"], rsuffix="2", how="inner")

# Add Some Diff Columns

In [84]:
for column in ma_columns:
    try:
        full_df[column + "diff"] = full_df[column] - full_df[str(column +"2")]
    except Exception as e:
        print(e)

In [85]:
full_df.to_pickle("../ncaa_data/full_df.p")

In [86]:
full_df.tail()

,Season,DayNum,WScore,LScore,Team1,Team2,Team1Score,Team2Score,Team1Home,Team2Home,index,Ast,Blk,DR,DayZero,FGA,FGA3,FGM,FGM3,FGM3_Perc,FGM_Perc,FTA,FTM,FTM_Perc,OR,PF,Score,Stl,TO,WFGM_Perc,Seed,7OT,ACU,ADE,AP,ARG,AUS,AWS,BBT,BCM,BD,BIH,BKM,BLS,BNM,BNT,BOB,BOW,BP5,BPI,BRZ,BUR,BWE,CJB,CMV,CNG,COL,COX,CPA,CPR,CRO,CRW,CTL,D1A,DAV,DC,DCI,DDB,DES,DII,DOK,DOL,DUN,DWH,EBB,EBP,ECK,ENT,ERD,ESR,FAS,FMG,FSH,GC,GRN,GRS,HAS,HAT,HER,HKB,HKS,HOL,HRN,IMS,INC,INP,ISR,JCI,JEN,JJK,JNG,JON,JRT,KBM,KEL,KLK,KMV,KOS,KPI,KPK,KRA,LAB,LAW,LEF,LMC,LOG,LYD,LYN,MAS,MB,MCL,MGS,MGY,MIC,MKV,MMG,MOR,MPI,MSX,MUZ,MvG,NET,NOL,NOR,OCT,OMY,PEQ,PGH,PH,PIG,PIR,PKL,PMC,POM,PPR,PRR,PTS,RAG,REI,REN,REW,RIS,RM,ROG,ROH,RPI,RSE,RSL,RT,RTB,RTH,RTP,RTR,SAG,SAP,SAU,SCR,SE,SEL,SFX,SGR,SIM,SMN,SMS,SP,SPR,SPW,STF,STH,STM,STR,STS,TBD,TMR,TOL,TPR,TRK,TRP,TRX,TSR,TW,UCS,UPS,USA,WIL,WLK,WLS,WMR,WMV,WOB,WOL,WTE,YAG,ZAM,date,index2,Ast2,Blk2,DR2,DayZero2,FGA2,FGA32,FGM2,FGM32,FGM3_Perc2,FGM_Perc2,FTA2,FTM2,FTM_Perc2,OR2,PF2,Score2,Stl2,TO2,WFGM_Perc2,Seed2,7OT2,ACU2,ADE2,AP2,ARG2,AUS2,AWS2,BBT2,BCM2,BD2,BIH2,BKM2,BLS2,BNM2,BNT2,BOB2,BOW2,BP52,BPI2,BRZ2,BUR2,BWE2,CJB2,CMV2,...,RAG2,REI2,REN2,REW2,RIS2,RM2,ROG2,ROH2,RPI2,RSE2,RSL2,RT2,RTB2,RTH2,RTP2,RTR2,SAG2,SAP2,SAU2,SCR2,SE2,SEL2,SFX2,SGR2,SIM2,SMN2,SMS2,SP2,SPR2,SPW2,STF2,STH2,STM2,STR2,STS2,TBD2,TMR2,TOL2,TPR2,TRK2,TRP2,TRX2,TSR2,TW2,UCS2,UPS2,USA2,WIL2,WLK2,WLS2,WMR2,WMV2,WOB2,WOL2,WTE2,YAG2,ZAM2,date2,indexdiff,Astdiff,Blkdiff,DRdiff,FGAdiff,FGA3diff,FGMdiff,FGM3diff,FGM3_Percdiff,FGM_Percdiff,FTAdiff,FTMdiff,FTM_Percdiff,ORdiff,PFdiff,Stldiff,TOdiff,WFGM_Percdiff,Seeddiff,7OTdiff,ACUdiff,ADEdiff,APdiff,ARGdiff,AUSdiff,AWSdiff,BBTdiff,BCMdiff,BDdiff,BIHdiff,BKMdiff,BLSdiff,BNMdiff,BNTdiff,BOBdiff,BOWdiff,BP5diff,BPIdiff,BRZdiff,BURdiff,BWEdiff,CJBdiff,CMVdiff,CNGdiff,COLdiff,COXdiff,CPAdiff,CPRdiff,CROdiff,CRWdiff,CTLdiff,D1Adiff,DAVdiff,DCdiff,DCIdiff,DDBdiff,DESdiff,DIIdiff,DOKdiff,DOLdiff,DUNdiff,DWHdiff,EBBdiff,EBPdiff,ECKdiff,ENTdiff,ERDdiff,ESRdiff,FASdiff,FMGdiff,FSHdiff,GCdiff,GRNdiff,GRSdiff,HASdiff,HATdiff,HERdiff,HKBdiff,HKSdiff,HOLdiff,HRNdiff,IMSdiff,INCdiff,INPdiff,ISRdiff,JCIdiff,JENdiff,JJKdiff,JNGdiff,JONdiff,JRTdiff,KBMdiff,KELdiff,KLKdiff,KMVdiff,KOSdiff,KPIdiff,KPKdiff,KRAdiff,LABdiff,LAWdiff,LEFdiff,LMCdiff,LOGdiff,LYDdiff,LYNdiff,MASdiff,MBdiff,MCLdiff,MGSdiff,MGYdiff,MICdiff,MKVdiff,MMGdiff,MORdiff,MPIdiff,MSXdiff,MUZdiff,MvGdiff,NETdiff,NOLdiff,NORdiff,OCTdiff,OMYdiff,PEQdiff,PGHdiff,PHdiff,PIGdiff,PIRdiff,PKLdiff,PMCdiff,POMdiff,PPRdiff,PRRdiff,PTSdiff,RAGdiff,REIdiff,RENdiff,REWdiff,RISdiff,RMdiff,ROGdiff,ROHdiff,RPIdiff,RSEdiff,RSLdiff,RTdiff,RTBdiff,RTHdiff,RTPdiff,RTRdiff,SAGdiff,SAPdiff,SAUdiff,SCRdiff,SEdiff,SELdiff,SFXdiff,SGRdiff,SIMdiff,SMNdiff,SMSdiff,SPdiff,SPRdiff,SPWdiff,STFdiff,STHdiff,STMdiff,STRdiff,STSdiff,TBDdiff,TMRdiff,TOLdiff,TPRdiff,TRKdiff,TRPdiff,TRXdiff,TSRdiff,TWdiff,UCSdiff,UPSdiff,USAdiff,WILdiff,WLKdiff,WLSdiff,WMRdiff,WMVdiff,WOBdiff,WOLdiff,WTEdiff,YAGdiff,ZAMdiff
88539,2019,131,82.0,78.0,1246,1397.0,78.0,82.0,False,False,88026.50,11.80,5.40,29.00,11/5/2018,48.45,12.50,22.60,4.80,0.375551,0.487902,26.30,19.25,0.727859,8.75,16.85,73.0,3.85,11.90,0.347727,2.0,135.0,274.0,275.0,10.0,117.0,242.0,77.0,33.0,284.0,313.0,79.0,226.0,327.0,115.0,132.0,278.0,279.0,228.0,192.0,291.0,127.0,36.0,198.0,330.0,141.0,75.0,316.0,308.0,285.0,296.0,278.0,292.0,66.0,105.0,262.0,19.0,269.0,8.0,20.0,43.0,74.0,126.0,174.0,121.0,23.0,238.0,315.0,265.0,81.0,47.0,115.0,39.0,248.0,259.0,187.0,132.0,209.0,320.0,270.0,283.0,266.0,255.0,273.0,138.0,283.0,268.0,170.0,200.0,74.0,32.0,253.0,275.0,275.0,282.0,233.0,282.0,165.0,88.0,27.0,85.0,332.0,140.0,20.0,153.0,71.0,222.0,202.0,22.0,177.0,250.0,70.0,254.0,237.0,232.0,144.0,76.0,255.0,276.0,295.0,292.0,61.0,282.0,264.0,291.0,191.0,253.0,27.0,271.0,20.0,145.0,198.0,280.0,19.0,186.0,330.0,170.0,286.0,197.0,170.0,70.0,308.0,268.0,183.0,181.0,329.0,243.0,266.0,76.0,214.0,106.0,13.0,276.0,48.0,189.0,247.0,180.0,288.0,93.0,122.0,113.0,230.0,79.0,102.0,53.0,129.0

# Match Ups to Predict

In [87]:
pairs_to_predict = pd.read_csv(DATA_PATH + "SampleSubmissionStage1.csv")["ID"].str.split("_")
seasons = pairs_to_predict.apply(lambda x: int(x[0]))
team1 = pairs_to_predict.apply(lambda x: int(x[1]))
team2 = pairs_to_predict.apply(lambda x:int(x[2]))

In [88]:
tourney_matchups = pd.DataFrame()
tourney_matchups["Team1"] = team1
tourney_matchups["Team2"] = team2
tourney_matchups["Season"] = seasons
tourney_matchups["DayNum"] = 149
tourney_matchups["Team1Home"] = 0
tourney_matchups["Team2Home"] = 0

In [89]:
last_games = team_df.reset_index()[team_df.reset_index()["DayNum"]==149]

In [90]:
last_games = last_games.set_index(["Season","TeamID","DayNum"])

In [91]:
tourney_matchups = tourney_matchups.join(last_games, on=["Season", "Team1","DayNum"], how="right").join(last_games, on=["Season", "Team2","DayNum"], rsuffix="2", how="inner")

In [92]:
for column in ma_columns:
    try:
        tourney_matchups[column + "diff"] = tourney_matchups[column] - tourney_matchups[str(column +"2")]
    except Exception as e:
        print(e)

In [93]:
tourney_matchups.to_pickle("../ncaa_data/tourney_matchups.p")

In [96]:
tourney_matchups.tail()

,Team1,Team2,Season,DayNum,Team1Home,Team2Home,index,Ast,Blk,DR,DayZero,FGA,FGA3,FGM,FGM3,FGM3_Perc,FGM_Perc,FTA,FTM,FTM_Perc,OR,PF,Score,Stl,TO,WFGM_Perc,Seed,7OT,ACU,ADE,AP,ARG,AUS,AWS,BBT,BCM,BD,BIH,BKM,BLS,BNM,BNT,BOB,BOW,BP5,BPI,BRZ,BUR,BWE,CJB,CMV,CNG,COL,COX,CPA,CPR,CRO,CRW,CTL,D1A,DAV,DC,DCI,DDB,DES,DII,DOK,DOL,DUN,DWH,EBB,EBP,ECK,ENT,ERD,ESR,FAS,FMG,FSH,GC,GRN,GRS,HAS,HAT,HER,HKB,HKS,HOL,HRN,IMS,INC,INP,ISR,JCI,JEN,JJK,JNG,JON,JRT,KBM,KEL,KLK,KMV,KOS,KPI,KPK,KRA,LAB,LAW,LEF,LMC,LOG,LYD,LYN,MAS,MB,MCL,MGS,MGY,MIC,MKV,MMG,MOR,MPI,MSX,MUZ,MvG,NET,NOL,NOR,OCT,OMY,PEQ,PGH,PH,PIG,PIR,PKL,PMC,POM,PPR,PRR,PTS,RAG,REI,REN,REW,RIS,RM,ROG,ROH,RPI,RSE,RSL,RT,RTB,RTH,RTP,RTR,SAG,SAP,SAU,SCR,SE,SEL,SFX,SGR,SIM,SMN,SMS,SP,SPR,SPW,STF,STH,STM,STR,STS,TBD,TMR,TOL,TPR,TRK,TRP,TRX,TSR,TW,UCS,UPS,USA,WIL,WLK,WLS,WMR,WMV,WOB,WOL,WTE,YAG,ZAM,date,index2,Ast2,Blk2,DR2,DayZero2,FGA2,FGA32,FGM2,FGM32,FGM3_Perc2,FGM_Perc2,FTA2,FTM2,FTM_Perc2,OR2,PF2,Score2,Stl2,TO2,WFGM_Perc2,Seed2,7OT2,ACU2,ADE2,AP2,ARG2,AUS2,AWS2,BBT2,BCM2,BD2,BIH2,BKM2,BLS2,BNM2,BNT2,BOB2,BOW2,BP52,BPI2,BRZ2,BUR2,BWE2,CJB2,CMV2,CNG2,COL2,COX2,CPA2,...,RAG2,REI2,REN2,REW2,RIS2,RM2,ROG2,ROH2,RPI2,RSE2,RSL2,RT2,RTB2,RTH2,RTP2,RTR2,SAG2,SAP2,SAU2,SCR2,SE2,SEL2,SFX2,SGR2,SIM2,SMN2,SMS2,SP2,SPR2,SPW2,STF2,STH2,STM2,STR2,STS2,TBD2,TMR2,TOL2,TPR2,TRK2,TRP2,TRX2,TSR2,TW2,UCS2,UPS2,USA2,WIL2,WLK2,WLS2,WMR2,WMV2,WOB2,WOL2,WTE2,YAG2,ZAM2,date2,indexdiff,Astdiff,Blkdiff,DRdiff,FGAdiff,FGA3diff,FGMdiff,FGM3diff,FGM3_Percdiff,FGM_Percdiff,FTAdiff,FTMdiff,FTM_Percdiff,ORdiff,PFdiff,Stldiff,TOdiff,WFGM_Percdiff,Seeddiff,7OTdiff,ACUdiff,ADEdiff,APdiff,ARGdiff,AUSdiff,AWSdiff,BBTdiff,BCMdiff,BDdiff,BIHdiff,BKMdiff,BLSdiff,BNMdiff,BNTdiff,BOBdiff,BOWdiff,BP5diff,BPIdiff,BRZdiff,BURdiff,BWEdiff,CJBdiff,CMVdiff,CNGdiff,COLdiff,COXdiff,CPAdiff,CPRdiff,CROdiff,CRWdiff,CTLdiff,D1Adiff,DAVdiff,DCdiff,DCIdiff,DDBdiff,DESdiff,DIIdiff,DOKdiff,DOLdiff,DUNdiff,DWHdiff,EBBdiff,EBPdiff,ECKdiff,ENTdiff,ERDdiff,ESRdiff,FASdiff,FMGdiff,FSHdiff,GCdiff,GRNdiff,GRSdiff,HASdiff,HATdiff,HERdiff,HKBdiff,HKSdiff,HOLdiff,HRNdiff,IMSdiff,INCdiff,INPdiff,ISRdiff,JCIdiff,JENdiff,JJKdiff,JNGdiff,JONdiff,JRTdiff,KBMdiff,KELdiff,KLKdiff,KMVdiff,KOSdiff,KPIdiff,KPKdiff,KRAdiff,LABdiff,LAWdiff,LEFdiff,LMCdiff,LOGdiff,LYDdiff,LYNdiff,MASdiff,MBdiff,MCLdiff,MGSdiff,MGYdiff,MICdiff,MKVdiff,MMGdiff,MORdiff,MPIdiff,MSXdiff,MUZdiff,MvGdiff,NETdiff,NOLdiff,NORdiff,OCTdiff,OMYdiff,PEQdiff,PGHdiff,PHdiff,PIGdiff,PIRdiff,PKLdiff,PMCdiff,POMdiff,PPRdiff,PRRdiff,PTSdiff,RAGdiff,REIdiff,RENdiff,REWdiff,RISdiff,RMdiff,ROGdiff,ROHdiff,RPIdiff,RSEdiff,RSLdiff,RTdiff,RTBdiff,RTHdiff,RTPdiff,RTRdiff,SAGdiff,SAPdiff,SAUdiff,SCRdiff,SEdiff,SELdiff,SFXdiff,SGRdiff,SIMdiff,SMNdiff,SMSdiff,SPdiff,SPRdiff,SPWdiff,STFdiff,STHdiff,STMdiff,STRdiff,STSdiff,TBDdiff,TMRdiff,TOLdiff,TPRdiff,TRKdiff,TRPdiff,TRXdiff,TSRdiff,TWdiff,UCSdiff,UPSdiff,USAdiff,WILdiff,WLKdiff,WLSdiff,WMRdiff,WMVdiff,WOBdiff,WOLdiff,WTEdiff,YAGdiff,ZAMdiff
11379,1438,1462.0,2018,149,0.0,0.0,83040.0,8.55,1.30,19.25,10/30/2017,54.40,20.30,22.65,5.55,0.290894,0.422333,12.20,8.75,0.616477,5.60,16.05,54.0,6.95,6.15,0.387456,1.0,1.0,274.0,275.0,1.0,1.0,242.0,176.788777,1.0,284.0,313.0,1.0,226.0,5.0,115.0,2.0,278.0,279.0,228.0,192.0,291.0,1.0,1.0,198.0,330.0,5.0,1.0,1.0,308.0,285.0,296.0,278.0,292.0,66.0,2.0,262.0,1.0,2.0,1.0,1.0,4.0,1.0,1.0,174.0,121.0,2.0,238.0,315.0,265.0,1.0,1.0,1.0,1.0,248.0,259.0,187.0,2.0,209.0,320.0,270.0,283.0,266.0,255.0,273.0,176.741681,283.0,268.0,170.0,200.0,1.0,1.0,253.0,275.0,275.0,282.0,233.0,282.0,165.0,1.0,2.0,1.0,1.0,176.764149,177.0,4.0,3.0,222.0,202.0,1.0,177.0,250.0,176.77282,254.0,237.0,232.0,4.0,1.0,255.0,276.0,295.0,292.0,176.957494,282.0,264.0,291.0,191.0,253.0,1.0,271.0,1.0,176.805674,198.0,280.0,1.0,186.0,1.0,170.0,286.0,197.0,170.0,1.0,308.0,268.0,183.0,181.0,1.0,243.0,266.0,2.0,214.0,1.0,1.0,276.0,3.0,189.0,247.0,180.0,288.0,1.0,3.0,1.0,230.0,3.0,2.0,4.0,2.0,2.0,284.0,1.0,2.0,222.0,283.0,175.0,297.0,298.0,2.0,1.0,5.0,28.0,209.0,201.0,253.0,1

# Final Matches to Predict

In [98]:
pairs_to_predict = pd.read_csv(DATA_PATH + "SampleSubmissionStage2.csv")["ID"].str.split("_")
seasons = pairs_to_predict.apply(lambda x: int(x[0]))
team1 = pairs_to_predict.apply(lambda x: int(x[1]))
team2 = pairs_to_predict.apply(lambda x:int(x[2]))

tourney_matchups = pd.DataFrame()
tourney_matchups["Team1"] = team1
tourney_matchups["Team2"] = team2
tourney_matchups["Season"] = seasons
tourney_matchups["DayNum"] = 149
tourney_matchups["Team1Home"] = 0
tourney_matchups["Team2Home"] = 0

last_games = team_df.reset_index()[team_df.reset_index()["DayNum"]==149]

last_games = last_games.set_index(["Season","TeamID","DayNum"])

tourney_matchups = tourney_matchups.join(last_games, on=["Season", "Team1","DayNum"], how="right").join(last_games, on=["Season", "Team2","DayNum"], rsuffix="2", how="inner")

In [99]:
for column in ma_columns:
    try:
        tourney_matchups[column + "diff"] = tourney_matchups[column] - tourney_matchups[str(column +"2")]
    except Exception as e:
        print(e)

In [100]:
tourney_matchups.to_pickle("../ncaa_data/final_matchups.p")

In [103]:
tourney_matchups.tail()

,Team1,Team2,Season,DayNum,Team1Home,Team2Home,index,Ast,Blk,DR,DayZero,FGA,FGA3,FGM,FGM3,FGM3_Perc,FGM_Perc,FTA,FTM,FTM_Perc,OR,PF,Score,Stl,TO,WFGM_Perc,Seed,7OT,ACU,ADE,AP,ARG,AUS,AWS,BBT,BCM,BD,BIH,BKM,BLS,BNM,BNT,BOB,BOW,BP5,BPI,BRZ,BUR,BWE,CJB,CMV,CNG,COL,COX,CPA,CPR,CRO,CRW,CTL,D1A,DAV,DC,DCI,DDB,DES,DII,DOK,DOL,DUN,DWH,EBB,EBP,ECK,ENT,ERD,ESR,FAS,FMG,FSH,GC,GRN,GRS,HAS,HAT,HER,HKB,HKS,HOL,HRN,IMS,INC,INP,ISR,JCI,JEN,JJK,JNG,JON,JRT,KBM,KEL,KLK,KMV,KOS,KPI,KPK,KRA,LAB,LAW,LEF,LMC,LOG,LYD,LYN,MAS,MB,MCL,MGS,MGY,MIC,MKV,MMG,MOR,MPI,MSX,MUZ,MvG,NET,NOL,NOR,OCT,OMY,PEQ,PGH,PH,PIG,PIR,PKL,PMC,POM,PPR,PRR,PTS,RAG,REI,REN,REW,RIS,RM,ROG,ROH,RPI,RSE,RSL,RT,RTB,RTH,RTP,RTR,SAG,SAP,SAU,SCR,SE,SEL,SFX,SGR,SIM,SMN,SMS,SP,SPR,SPW,STF,STH,STM,STR,STS,TBD,TMR,TOL,TPR,TRK,TRP,TRX,TSR,TW,UCS,UPS,USA,WIL,WLK,WLS,WMR,WMV,WOB,WOL,WTE,YAG,ZAM,date,index2,Ast2,Blk2,DR2,DayZero2,FGA2,FGA32,FGM2,FGM32,FGM3_Perc2,FGM_Perc2,FTA2,FTM2,FTM_Perc2,OR2,PF2,Score2,Stl2,TO2,WFGM_Perc2,Seed2,7OT2,ACU2,ADE2,AP2,ARG2,AUS2,AWS2,BBT2,BCM2,BD2,BIH2,BKM2,BLS2,BNM2,BNT2,BOB2,BOW2,BP52,BPI2,BRZ2,BUR2,BWE2,CJB2,CMV2,CNG2,COL2,COX2,CPA2,...,RAG2,REI2,REN2,REW2,RIS2,RM2,ROG2,ROH2,RPI2,RSE2,RSL2,RT2,RTB2,RTH2,RTP2,RTR2,SAG2,SAP2,SAU2,SCR2,SE2,SEL2,SFX2,SGR2,SIM2,SMN2,SMS2,SP2,SPR2,SPW2,STF2,STH2,STM2,STR2,STS2,TBD2,TMR2,TOL2,TPR2,TRK2,TRP2,TRX2,TSR2,TW2,UCS2,UPS2,USA2,WIL2,WLK2,WLS2,WMR2,WMV2,WOB2,WOL2,WTE2,YAG2,ZAM2,date2,indexdiff,Astdiff,Blkdiff,DRdiff,FGAdiff,FGA3diff,FGMdiff,FGM3diff,FGM3_Percdiff,FGM_Percdiff,FTAdiff,FTMdiff,FTM_Percdiff,ORdiff,PFdiff,Stldiff,TOdiff,WFGM_Percdiff,Seeddiff,7OTdiff,ACUdiff,ADEdiff,APdiff,ARGdiff,AUSdiff,AWSdiff,BBTdiff,BCMdiff,BDdiff,BIHdiff,BKMdiff,BLSdiff,BNMdiff,BNTdiff,BOBdiff,BOWdiff,BP5diff,BPIdiff,BRZdiff,BURdiff,BWEdiff,CJBdiff,CMVdiff,CNGdiff,COLdiff,COXdiff,CPAdiff,CPRdiff,CROdiff,CRWdiff,CTLdiff,D1Adiff,DAVdiff,DCdiff,DCIdiff,DDBdiff,DESdiff,DIIdiff,DOKdiff,DOLdiff,DUNdiff,DWHdiff,EBBdiff,EBPdiff,ECKdiff,ENTdiff,ERDdiff,ESRdiff,FASdiff,FMGdiff,FSHdiff,GCdiff,GRNdiff,GRSdiff,HASdiff,HATdiff,HERdiff,HKBdiff,HKSdiff,HOLdiff,HRNdiff,IMSdiff,INCdiff,INPdiff,ISRdiff,JCIdiff,JENdiff,JJKdiff,JNGdiff,JONdiff,JRTdiff,KBMdiff,KELdiff,KLKdiff,KMVdiff,KOSdiff,KPIdiff,KPKdiff,KRAdiff,LABdiff,LAWdiff,LEFdiff,LMCdiff,LOGdiff,LYDdiff,LYNdiff,MASdiff,MBdiff,MCLdiff,MGSdiff,MGYdiff,MICdiff,MKVdiff,MMGdiff,MORdiff,MPIdiff,MSXdiff,MUZdiff,MvGdiff,NETdiff,NOLdiff,NORdiff,OCTdiff,OMYdiff,PEQdiff,PGHdiff,PHdiff,PIGdiff,PIRdiff,PKLdiff,PMCdiff,POMdiff,PPRdiff,PRRdiff,PTSdiff,RAGdiff,REIdiff,RENdiff,REWdiff,RISdiff,RMdiff,ROGdiff,ROHdiff,RPIdiff,RSEdiff,RSLdiff,RTdiff,RTBdiff,RTHdiff,RTPdiff,RTRdiff,SAGdiff,SAPdiff,SAUdiff,SCRdiff,SEdiff,SELdiff,SFXdiff,SGRdiff,SIMdiff,SMNdiff,SMSdiff,SPdiff,SPRdiff,SPWdiff,STFdiff,STHdiff,STMdiff,STRdiff,STSdiff,TBDdiff,TMRdiff,TOLdiff,TPRdiff,TRKdiff,TRPdiff,TRXdiff,TSRdiff,TWdiff,UCSdiff,UPSdiff,USAdiff,WILdiff,WLKdiff,WLSdiff,WMRdiff,WMVdiff,WOBdiff,WOLdiff,WTEdiff,YAGdiff,ZAMdiff
2267,1438,1463.0,2019,149,0.0,0.0,88486.00,8.0,2.00,13.00,11/5/2018,53.00,24.00,22.00,5.00,0.208333,0.520833,13.00,10.0,0.769231,7.00,17.00,59.0,5.00,8.0,0.415094,1.0,4.0,274.0,275.0,2.0,1.0,242.0,4.0,1.0,284.0,313.0,1.0,226.0,327.0,115.0,3.0,278.0,279.0,228.0,192.0,291.0,2.0,2.0,198.0,330.0,4.0,2.0,316.0,308.0,285.0,296.0,278.0,292.0,66.0,2.0,262.0,1.0,269.0,1.0,1.0,3.0,1.0,3.0,174.0,121.0,2.0,238.0,315.0,265.0,28.0,2.0,2.0,3.0,248.0,259.0,187.0,2.0,209.0,320.0,270.0,283.0,266.0,255.0,273.0,3.0,283.0,268.0,170.0,200.0,1.0,1.0,253.0,275.0,275.0,282.0,233.0,282.0,165.0,2.0,2.0,1.0,332.0,2.0,3.0,3.0,3.0,222.0,202.0,1.0,177.0,250.0,3.0,254.0,237.0,232.0,2.0,4.0,255.0,276.0,295.0,292.0,2.0,282.0,264.0,291.0,191.0,253.0,1.0,271.0,4.0,3.0,198.0,280.0,1.0,186.0,330.0,170.0,286.0,197.0,170.0,2.0,308.0,268.0,183.0,181.0,329.0,243.0,266.0,3.0,214.0,1.0,2.0,276.0,3.0,189.0,247.0,180.0,288.0,2.0,5.0,2.0,230.0,2.0,3.0,3.0,2.0,4.0,284.0,4.0,1.0,222.0,283.0,175.0,297.0,298.0,2.0,1.0,3.0,28.0,209.0,201.0,253.0,193.0,2.0,1.0,2.0,3.0,244.0,3.0,1.